### Imports

In [1]:
import pandas as pd
import dataset_tools as dt

## Preparing data

### Loading dataset

In [2]:
aggregated_dataset = pd.read_csv('datasets/per_day_participant_dataset_co_w_avg.csv')

In [3]:
aggregated_dataset.tail()

,Unnamed: 0.1,Unnamed: 0,id,date,mood,mood_count,circumplex.arousal,circumplex.arousal_count,circumplex.valence,circumplex.valence_count,...,appCat.social,appCat.social_count,appCat.travel,appCat.travel_count,appCat.unknown,appCat.unknown_count,appCat.utilities,appCat.utilities_count,appCat.weather,appCat.weather_count
2149,2149,2149,AS14.33,2014-05-27,6.2,5,-0.600000,5,0.4,5,...,2010.364,40,0.000,0,0.000,0,56.173,3,0.0,0
2150,2150,2150,AS14.33,2014-05-28,8.2,5,0.000000,5,1.2,5,...,5361.211,54,0.000,0,0.000,0,30.666,6,0.0,0
2151,2151,2151,AS14.33,2014-05-29,7.0,3,-1.333333,3,1.0,3,...,1789.922,9,0.939,1,0.000,0,3.199,2,0.0,0
2152,2152,2152,AS14.33,2014-05-30,6.8,5,-0.800000,5,-0.4,5,...,3166.409,42,1052.648,28,8.072,3,232.825,15,0.0,0
2153,2153,2153,AS14.33,2014-05-31,7.0,1,-2.000000,1,1.0,1,...,0.000,0,0.000,0,0.000,0,0.000,0,0.0,0


In [4]:
# NOT NEEDED ANYMORE
# Pares column Unnamed: 0
# aggregated_dataset = dt.reformat_aggregated_data(aggregated_dataset)

In [5]:
# NOT NEEDED ANYMORE
# average the first 3 columns using the counts
# aggregated_dataset = dt.average_data(aggregated_dataset, dt.var_names[0:4])

In [6]:
aggregated_dataset.head()

,Unnamed: 0.1,Unnamed: 0,id,date,mood,mood_count,circumplex.arousal,circumplex.arousal_count,circumplex.valence,circumplex.valence_count,...,appCat.social,appCat.social_count,appCat.travel,appCat.travel_count,appCat.unknown,appCat.unknown_count,appCat.utilities,appCat.utilities_count,appCat.weather,appCat.weather_count
0,0,0,AS14.01,2014-02-17,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
1,1,1,AS14.01,2014-02-18,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
2,2,2,AS14.01,2014-02-19,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
3,3,3,AS14.01,2014-02-20,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
4,4,4,AS14.01,2014-02-21,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0


In [7]:
# save the dataset
aggregated_dataset.to_csv('datasets/aggregated_dataset.csv', index=False)

### Counting missing values

In [8]:
# count the number of rows where mood_count is 0
zero_mood_count = aggregated_dataset[aggregated_dataset['mood_count'] == 0].shape[0]
print('Number of rows where mood_count is 0:', zero_mood_count, 'out of', aggregated_dataset.shape[0])

Number of rows where mood_count is 0: 886 out of 2154


In [9]:
counts_of_empty_mood = aggregated_dataset[aggregated_dataset['mood_count'] == 0].groupby('id').size()
counts_of_empty_mood


id
AS14.01    31
AS14.02    27
AS14.03    34
AS14.05    24
AS14.06    36
AS14.07     2
AS14.08    16
AS14.09    32
AS14.12    35
AS14.13    24
AS14.14    34
AS14.15    26
AS14.16    25
AS14.17    22
AS14.19    32
AS14.20    31
AS14.23    38
AS14.24     7
AS14.25    51
AS14.26    35
AS14.27    46
AS14.28    43
AS14.29    46
AS14.30    31
AS14.31    47
AS14.32    50
AS14.33    61
dtype: int64

In [10]:
participants = aggregated_dataset['id'].unique()
current_participant = participants[0]
# count lenght of empty mood sequences
empty_mood_sequences = {}
"""
Dictionry with the form of:
{
    'id': {
        'length_sequence': count
    }
}

Example:

{
    'AS14.01': {
        1: 10,
        2: 5
    },
    'AS14.02': {
        1: 7,
        2: 3
    }
}
"""


for participant in participants:
    current_length = 0
    for index, row in aggregated_dataset[aggregated_dataset['id'] == participant].iterrows():
        if row['mood_count'] == 0:
            current_length += 1
            if index == len(aggregated_dataset[aggregated_dataset['id'] == participant]) - 1:
                
                if participant not in empty_mood_sequences:
                    empty_mood_sequences[participant] = {}
                if current_length not in empty_mood_sequences[participant]:
                    empty_mood_sequences[participant][current_length] = 0
                empty_mood_sequences[participant][current_length] += 1

                current_length = 0
        else:
            if current_length != 0:
                if participant not in empty_mood_sequences:
                    empty_mood_sequences[participant] = {}
                if current_length not in empty_mood_sequences[participant]:
                    empty_mood_sequences[participant][current_length] = 0
                empty_mood_sequences[participant][current_length] += 1
            current_length = 0

empty_mood_sequences

{'AS14.01': {9: 1, 21: 1, 1: 1},
 'AS14.02': {27: 1},
 'AS14.03': {32: 1, 1: 1},
 'AS14.05': {24: 1},
 'AS14.06': {35: 1, 1: 1},
 'AS14.07': {2: 1},
 'AS14.08': {16: 1},
 'AS14.09': {32: 1},
 'AS14.12': {24: 1, 11: 1},
 'AS14.13': {24: 1},
 'AS14.14': {31: 1, 1: 1, 2: 1},
 'AS14.15': {25: 1, 1: 1},
 'AS14.16': {24: 1, 1: 1},
 'AS14.17': {15: 1, 6: 1, 1: 1},
 'AS14.19': {32: 1},
 'AS14.20': {31: 1},
 'AS14.23': {33: 1, 1: 1},
 'AS14.24': {5: 1, 1: 1},
 'AS14.25': {50: 1, 1: 1},
 'AS14.26': {34: 1, 1: 1},
 'AS14.27': {45: 1, 1: 1},
 'AS14.28': {42: 1, 1: 1},
 'AS14.29': {44: 1, 1: 2},
 'AS14.30': {31: 1},
 'AS14.31': {44: 1, 1: 1},
 'AS14.32': {43: 1, 2: 1, 1: 3},
 'AS14.33': {58: 1, 2: 1, 1: 1}}

In [11]:
# plot mood_count per participant
import matplotlib.pyplot as plt
import os

if not os.path.exists('participant_mood_count'):
    os.makedirs('participant_mood_count')

for participant in participants:
    plt.plot(aggregated_dataset[aggregated_dataset['id'] == participant]['date'], aggregated_dataset[aggregated_dataset['id'] == participant]['mood_count'])
    plt.title('Participant ' + participant + ' mood count')
    plt.savefig('participant_mood_count/' + participant + '_mood_count.png')
    plt.close()

In [12]:
# plot per participant the sum of the variables that end with _count
for participant in participants:
    plt.plot(aggregated_dataset[aggregated_dataset['id'] == participant]['date'], aggregated_dataset[aggregated_dataset['id'] == participant].iloc[:, 3::2].sum(axis=1))
    plt.title('Participant ' + participant + ' sum of the variables that end with _count')
    plt.show()




TypeError: can only concatenate str (not "int") to str

In [ ]:
#Plot both
def plot_counts_per_participant(df):
    participants = df['id'].unique()
    for participant in participants:
        plt.plot(df[df['id'] == participant]['date'], df[df['id'] == participant]['mood_count'])
        plt.title('Participant ' + participant + ' mood count')
        plt.show()
        plt.plot(df[df['id'] == participant]['date'], df[df['id'] == participant].iloc[:, 3::2].sum(axis=1))
        plt.title('Participant ' + participant + ' sum of the _count variables')
        plt.show()


plot_counts_per_participant(aggregated_dataset)

## Cleaning

In [ ]:
#load the dataset
aggregated_dataset = pd.read_csv('datasets/aggregated_dataset.csv')

In [ ]:
aggregated_dataset['sum_of_counts'] = aggregated_dataset.iloc[:, 3::2].sum(axis=1)
aggregated_dataset.head()

In [ ]:
# strip the first rows and the last rows with mood_count = 0 per participant
aggregated_dataset_cleaned = pd.DataFrame()
for participant in participants:
    participant_data = aggregated_dataset[aggregated_dataset['id'] == participant]
    participant_data.reset_index(drop=True, inplace=True)
    first_non_zero_mood = participant_data[participant_data['mood_count'] != 0].index[0]
    last_non_zero_mood = participant_data[participant_data['mood_count'] != 0].index[-1]
    aggregated_dataset_cleaned = pd.concat([aggregated_dataset_cleaned, participant_data[first_non_zero_mood:last_non_zero_mood+1]])

In [ ]:
print('Original dataset shape:', aggregated_dataset.shape)
print('Cleaned dataset shape:', aggregated_dataset_cleaned.shape)

In [ ]:
# remove the period of 10 or more consecutive lower sum of the variables that end with _count per participant, splitting the dataset in batches. The treshold depends on the mean of the sum of the variables that end with _count
aggregated_dataset_cleaned = aggregated_dataset.copy(deep=True)

aggregated_dataset_cleaned.reset_index(drop=True, inplace=True)
aggregated_dataset_cleaned.head()




In [ ]:
#importing math
import math


def split_dataset(df, max_empty_lenght=4, min_count=10):
    participants = df['id'].unique()
    splitted_dataset = pd.DataFrame()
    for participant in participants:
        participant_data = df[df['id'] == participant]
        participant_data.reset_index(drop=True, inplace=True)
        mean_sum_of_counts = participant_data['sum_of_counts'].mean()
        #std_sum_of_counts = participant_data['sum_of_counts'].std()
        treshold = max(mean_sum_of_counts - 3*math.sqrt(mean_sum_of_counts), min_count) # 99% confidence interval assuming poisson distribution approximated by normal distribution
        print('threshold for participant', participant, 'is', treshold)
        current_length = 0
        batch_number = 0
        previous_cut = 0
        for index, row in participant_data.iterrows():
            if row['sum_of_counts'] <= treshold:
                current_length += 1

            elif current_length > max_empty_lenght:
                batch = participant_data[previous_cut:index-current_length+(max_empty_lenght if previous_cut else 0)].copy(deep=True)
                batch['batch'] = batch_number
                splitted_dataset = pd.concat([splitted_dataset, batch])

                batch_number += 1 if previous_cut else 0
                current_length = 0
                previous_cut = index
            else:
                current_length = 0
            if index == len(participant_data) - 1:
                if current_length <= max_empty_lenght:
                    batch = participant_data[previous_cut:].copy(deep=True)
                    batch['batch'] = batch_number
                    splitted_dataset = pd.concat([splitted_dataset, batch])
    return splitted_dataset

splitted_dataset = split_dataset(aggregated_dataset_cleaned)

In [ ]:
print('Original dataset shape:', aggregated_dataset_cleaned.shape)
print('Splitted dataset shape:', splitted_dataset.shape)

In [ ]:
# print lenght per batch
splitted_dataset.groupby(['id', 'batch']).size()

In [ ]:
#plot counts
splitted_dataset['date'] = pd.to_datetime(splitted_dataset['date'])
dt.plot_counts_per_participant(splitted_dataset)

In [ ]:
reordered_columns = splitted_dataset.columns.to_list()[0:2]+splitted_dataset.columns.to_list()[-1:] + splitted_dataset.columns.to_list()[2:-1]
splitted_dataset = splitted_dataset[reordered_columns]

In [ ]:
splitted_dataset[splitted_dataset['id'] == 'AS14.01']

In [ ]:
# save the dataset
splitted_dataset.to_csv('datasets/splitted_dataset.csv', index=False)

## Imputing missing days

In [2]:
# load the dataset
splitted_dataset = pd.read_csv('splitted_dataset.csv')

In [3]:
splitted_dataset.groupby(['participant_id', 'batch']).size()

participant_id  batch
AS14.01         0        46
AS14.03         0        49
AS14.05         0        53
AS14.06         0        45
AS14.08         0        54
AS14.12         0        40
AS14.13         0        52
AS14.14         0        40
                1         3
AS14.15         0        56
AS14.16         0        54
AS14.17         0        46
AS14.19         0        45
AS14.23         0        44
AS14.24         0        56
AS14.25         0        31
AS14.26         0        48
AS14.27         0        43
AS14.28         0        38
AS14.29         0        44
AS14.30         0        47
AS14.32         0        44
AS14.33         0        46
dtype: int64

In [4]:
# count the number of rows where mood_count is 0 per participant and batch
zero_mood_count = splitted_dataset[splitted_dataset['mood_count'] == 0].groupby(['participant_id', 'batch']).size()

In [5]:
zero_mood_count

participant_id  batch
AS14.01         0        1
AS14.03         0        2
AS14.06         0        1
AS14.14         0        3
AS14.15         0        1
AS14.16         0        1
AS14.17         0        1
AS14.23         0        5
AS14.24         0        2
AS14.25         0        1
AS14.26         0        1
AS14.27         0        1
AS14.28         0        1
AS14.29         0        2
AS14.32         0        7
AS14.33         0        3
dtype: int64

In [27]:
splitted_dataset.head()

,date_time,participant_id,batch,mood,mood_count,circumplex.arousal,circumplex.arousal_count,circumplex.valence,circumplex.valence_count,activity,...,appCat.social_count,appCat.travel,appCat.travel_count,appCat.unknown,appCat.unknown_count,appCat.utilities,appCat.utilities_count,appCat.weather,appCat.weather_count,sum_of_counts
0,2014-03-21,AS14.01,0,6.20,5,0.2,5,0.2,5,0.134050,...,60,915.445,13,0.000,0,598.754,26,0.000,0,632
1,2014-03-22,AS14.01,0,6.40,5,0.6,5,0.5,4,0.236880,...,12,37.305,1,0.000,0,117.621,3,0.000,0,381
2,2014-03-23,AS14.01,0,6.80,5,0.2,5,0.8,5,0.142741,...,18,0.000,0,0.000,0,30.086,1,30.386,1,368
3,2014-03-24,AS14.01,0,6.00,5,0.8,5,0.0,5,0.078961,...,43,419.805,5,0.000,0,178.732,2,0.000,0,685
4,2014-03-25,AS14.01,0,6.75,4,0.5,4,0.5,4,0.098374,...,28,0.000,0,235.223,8,222.893,10,0.000,0,473


In [29]:
dataset_for_knn = splitted_dataset.copy(deep=True)
dataset_for_knn['date_time'] = pd.to_datetime(dataset_for_knn['date_time'])

In [30]:
columns_to_remove = dt.count_var_names.copy()
columns_to_remove.remove('mood_count')

In [31]:
dataset_for_knn.drop(dataset_for_knn.columns[-1], axis=1, inplace=True)
dataset_for_knn.head()

,date_time,participant_id,batch,mood,mood_count,circumplex.arousal,circumplex.arousal_count,circumplex.valence,circumplex.valence_count,activity,...,appCat.social,appCat.social_count,appCat.travel,appCat.travel_count,appCat.unknown,appCat.unknown_count,appCat.utilities,appCat.utilities_count,appCat.weather,appCat.weather_count
0,2014-03-21,AS14.01,0,6.20,5,0.2,5,0.2,5,0.134050,...,4508.500,60,915.445,13,0.000,0,598.754,26,0.000,0
1,2014-03-22,AS14.01,0,6.40,5,0.6,5,0.5,4,0.236880,...,439.632,12,37.305,1,0.000,0,117.621,3,0.000,0
2,2014-03-23,AS14.01,0,6.80,5,0.2,5,0.8,5,0.142741,...,900.839,18,0.000,0,0.000,0,30.086,1,30.386,1
3,2014-03-24,AS14.01,0,6.00,5,0.8,5,0.0,5,0.078961,...,3223.626,43,419.805,5,0.000,0,178.732,2,0.000,0
4,2014-03-25,AS14.01,0,6.75,4,0.5,4,0.5,4,0.098374,...,1919.471,28,0.000,0,235.223,8,222.893,10,0.000,0


In [32]:

dataset_for_knn.drop(columns_to_remove, axis=1, inplace=True)

In [33]:
dataset_for_knn.head()

,date_time,participant_id,batch,mood,mood_count,circumplex.arousal,circumplex.valence,activity,screen,call,...,appCat.entertainment,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather
0,2014-03-21,AS14.01,0,6.20,5,0.2,0.2,0.134050,17978.907000,6.0,...,1007.456,49.544,0.0,172.206,239.751,4508.500,915.445,0.000,598.754,0.000
1,2014-03-22,AS14.01,0,6.40,5,0.6,0.5,0.236880,6142.161000,3.0,...,93.324,21.076,0.0,0.000,98.143,439.632,37.305,0.000,117.621,0.000
2,2014-03-23,AS14.01,0,6.80,5,0.2,0.8,0.142741,6773.832001,0.0,...,94.346,43.403,0.0,0.000,72.823,900.839,0.000,0.000,30.086,30.386
3,2014-03-24,AS14.01,0,6.00,5,0.8,0.0,0.078961,15047.351001,10.0,...,976.971,34.106,0.0,3.010,66.558,3223.626,419.805,0.000,178.732,0.000
4,2014-03-25,AS14.01,0,6.75,4,0.5,0.5,0.098374,13569.968462,0.0,...,68.206,43.054,0.0,0.000,178.819,1919.471,0.000,235.223,222.893,0.000


In [34]:
# predict the mood of days where the mood_count == 0 using knn
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

def predict_mood(df, k=5):
    partecipants = df['participant_id'].unique()
    for participant in partecipants:
        participant_data = df[df['participant_id'] == participant]
        participant_data.reset_index(drop=True, inplace=True)


        X = participant_data[participant_data['mood_count'] != 0].iloc[:, 3:].drop('mood_count', axis=1)
        #y = participant_data[participant_data['mood_count'] != 0]['mood']
        y = participant_data[participant_data['mood_count'] != 0]['mood'].round().astype(int)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print('Accuracy for participant', participant, 'is', accuracy_score(y_test, y_pred))


predict_mood(splitted_dataset)

        


Accuracy for participant AS14.01 is 0.5555555555555556
Accuracy for participant AS14.03 is 0.8
Accuracy for participant AS14.05 is 0.6363636363636364
Accuracy for participant AS14.06 is 0.6666666666666666
Accuracy for participant AS14.08 is 0.5454545454545454
Accuracy for participant AS14.12 is 0.75
Accuracy for participant AS14.13 is 0.6363636363636364
Accuracy for participant AS14.14 is 0.5
Accuracy for participant AS14.15 is 0.9090909090909091
Accuracy for participant AS14.16 is 0.6363636363636364
Accuracy for participant AS14.17 is 0.7777777777777778
Accuracy for participant AS14.19 is 0.6666666666666666
Accuracy for participant AS14.23 is 0.25
Accuracy for participant AS14.24 is 0.6363636363636364
Accuracy for participant AS14.25 is 0.16666666666666666
Accuracy for participant AS14.26 is 0.7
Accuracy for participant AS14.27 is 0.4444444444444444
Accuracy for participant AS14.28 is 0.25
Accuracy for participant AS14.29 is 0.6666666666666666
Accuracy for participant AS14.30 is 0.9
A

In [35]:
# do the above using all the values for training and using the predctions to fill the mood_count == 0
def predict_mood_fill(df, k=5):
    partecipants = df['participant_id'].unique()
    for participant in partecipants:
        print('Predicting mood for participant', participant)
        participant_data = df[df['participant_id'] == participant]
        #participant_data.reset_index(drop=True, inplace=True)

        test_indices = participant_data[participant_data['mood_count'] == 0].index

        train_indices = participant_data[participant_data['mood_count'] != 0].index

        X_train = participant_data.loc[train_indices].iloc[:, 3:].drop('mood_count', axis=1)
        y_train = participant_data.loc[train_indices]['mood'].round().astype(int)
        X_test = participant_data.loc[test_indices].iloc[:, 3:].drop('mood_count', axis=1)
        


        if X_test.empty:
            print('No mood to predict for participant', participant)
            continue
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        # print predictions
        print('Predictions for participant', participant, 'are', y_pred)
        
        df.loc[test_indices,'mood'] = y_pred

    return df

In [36]:
splitted_dataset.head()

,date_time,participant_id,batch,mood,mood_count,circumplex.arousal,circumplex.arousal_count,circumplex.valence,circumplex.valence_count,activity,...,appCat.social_count,appCat.travel,appCat.travel_count,appCat.unknown,appCat.unknown_count,appCat.utilities,appCat.utilities_count,appCat.weather,appCat.weather_count,sum_of_counts
0,2014-03-21,AS14.01,0,6.20,5,0.2,5,0.2,5,0.134050,...,60,915.445,13,0.000,0,598.754,26,0.000,0,632
1,2014-03-22,AS14.01,0,6.40,5,0.6,5,0.5,4,0.236880,...,12,37.305,1,0.000,0,117.621,3,0.000,0,381
2,2014-03-23,AS14.01,0,6.80,5,0.2,5,0.8,5,0.142741,...,18,0.000,0,0.000,0,30.086,1,30.386,1,368
3,2014-03-24,AS14.01,0,6.00,5,0.8,5,0.0,5,0.078961,...,43,419.805,5,0.000,0,178.732,2,0.000,0,685
4,2014-03-25,AS14.01,0,6.75,4,0.5,4,0.5,4,0.098374,...,28,0.000,0,235.223,8,222.893,10,0.000,0,473


In [37]:
filled_dataset = dataset_for_knn.copy(deep=True)
print('Before filling mood_count == 0')
print(filled_dataset[filled_dataset['mood'] == 0]['mood'])
print(f"shape of dataset: {filled_dataset.shape}")
filled_dataset= predict_mood_fill(filled_dataset)
print('After filling mood_count == 0')
print(filled_dataset[filled_dataset['mood'] == 0]['mood'])
print(f"shape of dataset: {filled_dataset.shape}")


Before filling mood_count == 0
45     0.0
92     0.0
94     0.0
190    0.0
362    0.0
377    0.0
378    0.0
435    0.0
489    0.0
536    0.0
606    0.0
623    0.0
624    0.0
625    0.0
626    0.0
648    0.0
682    0.0
711    0.0
737    0.0
795    0.0
840    0.0
873    0.0
877    0.0
952    0.0
953    0.0
955    0.0
959    0.0
969    0.0
976    0.0
977    0.0
994    0.0
995    0.0
998    0.0
Name: mood, dtype: float64
shape of dataset: (1024, 23)
Predicting mood for participant AS14.01
Predictions for participant AS14.01 are [6]
Predicting mood for participant AS14.03
Predictions for participant AS14.03 are [7 7]
Predicting mood for participant AS14.05
No mood to predict for participant AS14.05
Predicting mood for participant AS14.06
Predictions for participant AS14.06 are [6]
Predicting mood for participant AS14.08
No mood to predict for participant AS14.08
Predicting mood for participant AS14.12
No mood to predict for participant AS14.12
Predicting mood for participant AS14.13
No mood

In [40]:
splitted_dataset.head()

,date_time,participant_id,batch,mood,mood_count,circumplex.arousal,circumplex.arousal_count,circumplex.valence,circumplex.valence_count,activity,...,appCat.social_count,appCat.travel,appCat.travel_count,appCat.unknown,appCat.unknown_count,appCat.utilities,appCat.utilities_count,appCat.weather,appCat.weather_count,sum_of_counts
0,2014-03-21,AS14.01,0,6.20,5,0.2,5,0.2,5,0.134050,...,60,915.445,13,0.000,0,598.754,26,0.000,0,632
1,2014-03-22,AS14.01,0,6.40,5,0.6,5,0.5,4,0.236880,...,12,37.305,1,0.000,0,117.621,3,0.000,0,381
2,2014-03-23,AS14.01,0,6.80,5,0.2,5,0.8,5,0.142741,...,18,0.000,0,0.000,0,30.086,1,30.386,1,368
3,2014-03-24,AS14.01,0,6.00,5,0.8,5,0.0,5,0.078961,...,43,419.805,5,0.000,0,178.732,2,0.000,0,685
4,2014-03-25,AS14.01,0,6.75,4,0.5,4,0.5,4,0.098374,...,28,0.000,0,235.223,8,222.893,10,0.000,0,473


In [41]:
filled_dataset.head()

,date_time,participant_id,batch,mood,mood_count,circumplex.arousal,circumplex.valence,activity,screen,call,...,appCat.entertainment,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather
0,2014-03-21,AS14.01,0,6.20,5,0.2,0.2,0.134050,17978.907000,6.0,...,1007.456,49.544,0.0,172.206,239.751,4508.500,915.445,0.000,598.754,0.000
1,2014-03-22,AS14.01,0,6.40,5,0.6,0.5,0.236880,6142.161000,3.0,...,93.324,21.076,0.0,0.000,98.143,439.632,37.305,0.000,117.621,0.000
2,2014-03-23,AS14.01,0,6.80,5,0.2,0.8,0.142741,6773.832001,0.0,...,94.346,43.403,0.0,0.000,72.823,900.839,0.000,0.000,30.086,30.386
3,2014-03-24,AS14.01,0,6.00,5,0.8,0.0,0.078961,15047.351001,10.0,...,976.971,34.106,0.0,3.010,66.558,3223.626,419.805,0.000,178.732,0.000
4,2014-03-25,AS14.01,0,6.75,4,0.5,0.5,0.098374,13569.968462,0.0,...,68.206,43.054,0.0,0.000,178.819,1919.471,0.000,235.223,222.893,0.000


In [42]:
filled_complete_dataset = pd.concat([filled_dataset.iloc[:, 0:4], filled_dataset.iloc[:, 4:]], axis=1)

In [38]:
# save the dataset
filled_dataset.to_csv('filled_dataset.csv', index=False)

In [44]:
filled_complete_dataset.to_csv('filled_complete_dataset.csv', index=False)